In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

In [2]:
data = pd.read_csv('mpst_full_data.csv')
data.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [3]:
import spacy

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ankan_rokr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from tqdm import tqdm

In [6]:
# function for cleaning the plots of the movies
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    #text = re.sub('\W', ' ', text)
    #text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

# function for text cleaning 
def cleaned(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [7]:
data['cleaned'] = list(data['plot_synopsis'].apply(clean_text))

In [8]:
data['cleaned'] = list(data['cleaned'].apply(cleaned))

In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

data['cleaned'] = data['cleaned'].apply(lambda x: remove_stopwords(x))

In [10]:
data.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,cleaned
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb,note synopsis orginal italian release segments...
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb,two thousand years ago nhagruul foul sorcerer ...
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb,matuschek gift store budapest workplace alfred...
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb,glenn holland morning person anyone standards ...
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb,may cuban man named tony montana al pacino cla...


In [11]:
train_data =  data['split']=='train'
train = data[train_data]

   
    
test_data =  data['split']=='test'
test = data[test_data]

  
    
validation_data =  data['split']=='val'
val = data[validation_data]


# BoW


In [12]:
vectorizer_tags = CountVectorizer(tokenizer = lambda x: x.split(','), binary='true')
y_train = vectorizer_tags.fit_transform(train['tags'])
y_test = vectorizer_tags.transform(test['tags'])

In [13]:
y_train.shape

(9489, 142)

# Uni

In [14]:

start = datetime.now()
vectorizer_unigram = TfidfVectorizer(min_df=5, max_features=20000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,1))
x_train_multilabel_unigram = vectorizer_unigram.fit_transform(train['cleaned'])
x_test_multilabel_unigram = vectorizer_unigram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:03.718197


In [15]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_unigram,y_train)
predictions = classifier.predict (x_test_multilabel_unigram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0030343897505057315
Hamming loss  0.055070137615985865
Micro-average quality numbers
Precision: 0.1611, Recall: 0.3732, F1-measure: 0.2250
Macro-average quality numbers
Precision: 0.0726, Recall: 0.1633, F1-measure: 0.0967
              precision    recall  f1-score   support

           0       0.05      0.09      0.07        35
           1       0.14      0.43      0.22       117
           2       0.05      0.08      0.06        26
           3       0.06      0.18      0.09        11
           4       0.11      0.31      0.16        13
           5       0.03      0.08      0.04        26
           6       0.04      0.07      0.05        15
           7       0.05      0.19      0.08        75
           8       0.00      0.00      0.00         4
           9       0.03      0.03      0.03        31
          10       0.05      0.12      0.07         8
          11       0.02      0.05      0.03        20
          12       0.06      0.18      0.09        79
       

In [16]:

start = datetime.now()
vectorizer_bigram = TfidfVectorizer(min_df=5, max_features=20000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(2,2))
x_train_multilabel_bigram = vectorizer_bigram.fit_transform(train['cleaned'])
x_test_multilabel_bigram = vectorizer_bigram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:26.864846


In [17]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_bigram,y_train)
predictions = classifier.predict (x_test_multilabel_bigram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.02191503708698584
Hamming loss  0.03362284292403104
Micro-average quality numbers
Precision: 0.2403, Recall: 0.2635, F1-measure: 0.2513
Macro-average quality numbers
Precision: 0.0809, Recall: 0.0778, F1-measure: 0.0742
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.24      0.33      0.28       117
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        11
           4       0.13      0.15      0.14        13
           5       0.25      0.15      0.19        26
           6       0.00      0.00      0.00        15
           7       0.09      0.11      0.10        75
           8       0.00      0.00      0.00         4
           9       0.09      0.03      0.05        31
          10       0.40      0.25      0.31         8
          11       0.00      0.00      0.00        20
          12       0.07      0.06      0.07        79
          

In [18]:
start = datetime.now()
vectorizer_trigram = TfidfVectorizer(min_df=10, max_features=20000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(3,3))
x_train_multilabel_trigram = vectorizer_trigram.fit_transform(train['cleaned'])
x_test_multilabel_trigram = vectorizer_trigram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:35.543909


In [19]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_trigram,y_train)
predictions = classifier.predict (x_test_multilabel_trigram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.00033715441672285906
Hamming loss  0.11498865071752158
Micro-average quality numbers
Precision: 0.0565, Recall: 0.2781, F1-measure: 0.0939
Macro-average quality numbers
Precision: 0.0289, Recall: 0.1425, F1-measure: 0.0442
              precision    recall  f1-score   support

           0       0.02      0.14      0.03        35
           1       0.08      0.36      0.13       117
           2       0.01      0.08      0.02        26
           3       0.00      0.00      0.00        11
           4       0.01      0.08      0.01        13
           5       0.02      0.19      0.03        26
           6       0.01      0.07      0.01        15
           7       0.03      0.20      0.05        75
           8       0.00      0.00      0.00         4
           9       0.01      0.13      0.02        31
          10       0.01      0.25      0.02         8
          11       0.01      0.15      0.02        20
          12       0.03      0.20      0.06        79
       

In [20]:
start = datetime.now()
vectorizer_fourgram = TfidfVectorizer(min_df=10, max_features=20000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(4,4))
x_train_multilabel_fourgram = vectorizer_fourgram.fit_transform(train['cleaned'])
x_test_multilabel_fourgram = vectorizer_fourgram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:39.402924


In [21]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_fourgram,y_train)
predictions = classifier.predict (x_test_multilabel_fourgram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.3896080461189253
Micro-average quality numbers
Precision: 0.0185, Recall: 0.3303, F1-measure: 0.0350
Macro-average quality numbers
Precision: 0.0307, Recall: 0.3675, F1-measure: 0.0210
              precision    recall  f1-score   support

           0       0.01      0.94      0.02        35
           1       0.05      0.01      0.01       117
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        11
           4       0.00      1.00      0.01        13
           5       0.00      0.00      0.00        26
           6       0.00      0.00      0.00        15
           7       0.02      0.01      0.02        75
           8       0.00      0.00      0.00         4
           9       0.01      1.00      0.02        31
          10       0.00      0.00      0.00         8
          11       0.01      0.95      0.01        20
          12       0.03      0.96      0.05        79
          13       0.00    

In [22]:
start = datetime.now()
vectorizer_ngram = TfidfVectorizer(min_df=10, max_features=20000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,4))
x_train_multilabel_ngram = vectorizer_ngram.fit_transform(train['cleaned'])
x_test_multilabel_ngram = vectorizer_ngram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:02:15.210225


In [23]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_ngram,y_train)
predictions = classifier.predict (x_test_multilabel_ngram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0070802427511800405
Hamming loss  0.05429135839989363
Micro-average quality numbers
Precision: 0.1686, Recall: 0.3904, F1-measure: 0.2355
Macro-average quality numbers
Precision: 0.0725, Recall: 0.1672, F1-measure: 0.0988
              precision    recall  f1-score   support

           0       0.05      0.09      0.07        35
           1       0.14      0.48      0.22       117
           2       0.04      0.08      0.06        26
           3       0.07      0.18      0.10        11
           4       0.14      0.38      0.20        13
           5       0.03      0.08      0.04        26
           6       0.08      0.13      0.10        15
           7       0.06      0.24      0.10        75
           8       0.00      0.00      0.00         4
           9       0.03      0.03      0.03        31
          10       0.04      0.12      0.06         8
          11       0.04      0.10      0.06        20
          12       0.06      0.19      0.09        79
        

In [24]:
from scipy.sparse import coo_matrix, hstack

In [25]:
x_train_1 = hstack((x_train_multilabel_unigram, x_train_multilabel_bigram, x_train_multilabel_trigram),format="csr",dtype='float64')

In [26]:
x_test_1 = hstack((x_test_multilabel_unigram, x_test_multilabel_bigram, x_test_multilabel_trigram),format="csr",dtype='float64')

In [29]:
from sklearn.model_selection import GridSearchCV
start = datetime.now()

model = OneVsRestClassifier(SGDClassifier(loss='log', penalty='l2', class_weight="balanced", n_jobs = -1 ), n_jobs = -1)

param_grid = {
    "estimator__alpha": [10**-4,10**-3, 10**-2,10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]
}

model = GridSearchCV(model, param_grid, scoring='f1_micro',n_jobs=-1)

model.fit(x_train_1, y_train)

print (model.best_score_)
print (model.best_params_)
print("Time taken to run this cell :", datetime.now() - start)

0.18712397860903204
{'estimator__alpha': 0.001}
Time taken to run this cell : 0:01:18.256507


In [ ]:
# train model

start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_1, y_train)
predictions = classifier.predict(x_test_1)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.020903573836817263
Hamming loss  0.03734341314237414
Micro-average quality numbers
Precision: 0.2179, Recall: 0.2870, F1-measure: 0.2477
Macro-average quality numbers
Precision: 0.0793, Recall: 0.0968, F1-measure: 0.0842
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.20      0.36      0.26       117
           2       0.09      0.04      0.05        26
           3       0.00      0.00      0.00        11
           4       0.25      0.31      0.28        13
           5       0.18      0.12      0.14        26
           6       0.00      0.00      0.00        15
           7       0.06      0.09      0.07        75
           8       0.00      0.00      0.00         4
           9       0.05      0.03      0.04        31
          10       0.14      0.12      0.13         8
          11       0.00      0.00      0.00        20
          12       0.06      0.06      0.06        79
         

In [27]:
x_train_2 = hstack((x_train_multilabel_unigram, x_train_multilabel_bigram, x_train_multilabel_trigram, x_train_multilabel_fourgram),format="csr",dtype='float64')
x_test_2 = hstack((x_test_multilabel_unigram, x_test_multilabel_bigram, x_test_multilabel_trigram, x_test_multilabel_fourgram),format="csr",dtype='float64')

In [ ]:
start = datetime.now()

model = OneVsRestClassifier(SGDClassifier(loss='log', penalty='l2', class_weight="balanced", n_jobs = -1 ), n_jobs = -1)

param_grid = {
    "estimator__alpha": [10**-4,10**-3, 10**-2,10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]
}

model = GridSearchCV(model, param_grid, scoring='f1_micro',n_jobs=-1)

model.fit(x_train_2, y_train)

print (model.best_score_)
print (model.best_params_)
print("Time taken to run this cell :", datetime.now() - start)

0.19961240026772956
{'estimator__alpha': 0.001}
Time taken to run this cell : 0:01:09.366789


In [ ]:


start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_2, y_train)
predictions = classifier.predict(x_test_2)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.02191503708698584
Hamming loss  0.037549979580788845
Micro-average quality numbers
Precision: 0.2165, Recall: 0.2875, F1-measure: 0.2470
Macro-average quality numbers
Precision: 0.0844, Recall: 0.0979, F1-measure: 0.0849
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.20      0.34      0.25       117
           2       0.08      0.04      0.05        26
           3       0.00      0.00      0.00        11
           4       0.16      0.23      0.19        13
           5       0.21      0.15      0.18        26
           6       0.00      0.00      0.00        15
           7       0.06      0.09      0.07        75
           8       0.00      0.00      0.00         4
           9       0.05      0.03      0.04        31
          10       0.11      0.12      0.12         8
          11       0.00      0.00      0.00        20
          12       0.05      0.06      0.06        79
         

In [28]:
train_glove = np.load('glove_tarin.npy') 

In [29]:
test_glove = np.load('glove_test.npy') 

In [30]:
x_train_2 = hstack((x_train_multilabel_unigram, x_train_multilabel_bigram, x_train_multilabel_trigram, x_train_multilabel_fourgram),format="csr",dtype='float64')
x_test_2 = hstack((x_test_multilabel_unigram, x_test_multilabel_bigram, x_test_multilabel_trigram, x_test_multilabel_fourgram),format="csr",dtype='float64')

In [31]:
x_train_2 = x_train_2.toarray()

In [32]:
x_train_2.shape

(9489, 40857)

In [33]:
x_test_2 = x_test_2.toarray()

In [34]:
x_test_2.shape

(2966, 40857)

In [35]:
train_glove.shape

(9489, 300)

In [36]:
test_glove.shape

(2966, 300)

In [37]:
x_train_3 = np.hstack((x_train_2, train_glove))

In [38]:
x_train_3.shape

(9489, 41157)

In [39]:
x_test_3 = np.hstack((x_test_2, test_glove))

In [40]:
x_test_3.shape

(2966, 41157)

In [ ]:
np.save('gove+train2_for_every_feature.npy', x_train_3)
np.save('gove+test2_for_every_feature.npy', x_test_3)

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_3, y_train)
predictions = classifier.predict(x_test_3)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.009440323668240054
Hamming loss  0.0726781457456811
Micro-average quality numbers
Precision: 0.1803, Recall: 0.6750, F1-measure: 0.2846
Macro-average quality numbers
Precision: 0.1655, Recall: 0.6215, F1-measure: 0.2297
              precision    recall  f1-score   support

           0       0.05      0.49      0.10        35
           1       0.21      0.62      0.32       117
           2       0.87      1.00      0.93        26
           3       0.02      0.27      0.03        11
           4       0.11      0.85      0.20        13
           5       0.19      1.00      0.32        26
           6       0.14      1.00      0.24        15
           7       0.10      0.72      0.17        75
           8       0.02      0.25      0.03         4
           9       0.32      1.00      0.49        31
          10       0.29      0.88      0.44         8
          11       0.04      0.50      0.07        20
          12       0.12      0.47      0.19        79
          

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_3, y_train)
predictions = classifier.predict(x_test_3)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.08203774230005793
Micro-average quality numbers
Precision: 0.1569, Recall: 0.6471, F1-measure: 0.2526
Macro-average quality numbers
Precision: 0.1789, Recall: 0.5978, F1-measure: 0.2350
              precision    recall  f1-score   support

           0       0.05      0.17      0.08        35
           1       0.33      0.50      0.40       117
           2       0.93      1.00      0.96        26
           3       0.01      0.18      0.02        11
           4       0.16      0.69      0.26        13
           5       0.33      1.00      0.49        26
           6       0.11      0.93      0.20        15
           7       0.13      0.67      0.21        75
           8       0.00      0.25      0.01         4
           9       0.53      1.00      0.70        31
          10       0.09      0.75      0.16         8
          11       0.00      0.00      0.00        20
          12       0.13      0.42      0.20        79
          13       0.02   

In [41]:
x_train_1 = x_train_1.toarray()

In [42]:
x_test_1 = x_test_1.toarray()

In [43]:
x_train_4 = np.hstack((x_train_1, train_glove))

In [44]:
x_test_4 = np.hstack((x_test_1, test_glove))

In [ ]:
np.save('gove+train1_for_every_feature.npy', x_train_4)
np.save('gove+test1_for_every_feature.npy', x_test_4)

In [51]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_4, y_train)
predictions = classifier.predict(x_test_4)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.009103169251517195
Hamming loss  0.07314588814071211
Micro-average quality numbers
Precision: 0.1795, Recall: 0.6759, F1-measure: 0.2836
Macro-average quality numbers
Precision: 0.1697, Recall: 0.6251, F1-measure: 0.2363
              precision    recall  f1-score   support

           0       0.06      0.63      0.11        35
           1       0.18      0.73      0.29       117
           2       0.43      1.00      0.60        26
           3       0.02      0.27      0.03        11
           4       0.12      0.77      0.21        13
           5       0.19      1.00      0.31        26
           6       0.07      0.80      0.12        15
           7       0.11      0.76      0.19        75
           8       0.01      0.25      0.02         4
           9       0.25      1.00      0.41        31
          10       0.30      0.75      0.43         8
          11       0.02      0.60      0.04        20
          12       0.09      0.66      0.15        79
         

In [52]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.001, penalty='l2', class_weight="balanced", n_jobs = -1))
classifier.fit(x_train_4, y_train)
predictions = classifier.predict(x_test_4)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.008766014834794335
Hamming loss  0.0738344429354278
Micro-average quality numbers
Precision: 0.1736, Recall: 0.6505, F1-measure: 0.2740
Macro-average quality numbers
Precision: 0.1951, Recall: 0.6011, F1-measure: 0.2510
              precision    recall  f1-score   support

           0       0.05      0.63      0.09        35
           1       0.19      0.70      0.30       117
           2       0.90      1.00      0.95        26
           3       0.01      0.36      0.02        11
           4       0.14      0.77      0.23        13
           5       0.33      1.00      0.50        26
           6       0.09      0.67      0.16        15
           7       0.19      0.47      0.27        75
           8       0.03      0.25      0.05         4
           9       0.59      0.97      0.73        31
          10       0.88      0.88      0.88         8
          11       0.01      0.10      0.03        20
          12       0.13      0.46      0.21        79
          

In [45]:
x_train_5 = np.hstack((x_train_4, x_train_3))

In [46]:
x_test_5 = np.hstack((x_test_4, x_test_3))

In [ ]:
np.save('all1.npy_for_every_feature', x_train_5)
np.save('all2_for_every_feature.npy', x_test_5)

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_5, y_train)
predictions = classifier.predict(x_test_5)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0033715441672285905
Hamming loss  0.06653813643831974
Micro-average quality numbers
Precision: 0.1892, Recall: 0.6409, F1-measure: 0.2921
Macro-average quality numbers
Precision: 0.1661, Recall: 0.5899, F1-measure: 0.2263
              precision    recall  f1-score   support

           0       0.08      0.54      0.15        35
           1       0.21      0.68      0.31       117
           2       0.58      1.00      0.73        26
           3       0.01      0.18      0.02        11
           4       0.10      0.85      0.18        13
           5       0.19      1.00      0.32        26
           6       0.08      0.80      0.14        15
           7       0.12      0.73      0.20        75
           8       0.01      0.25      0.01         4
           9       0.30      1.00      0.46        31
          10       0.09      0.88      0.16         8
          11       0.03      0.20      0.05        20
          12       0.11      0.49      0.18        79
        

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.0001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_5, y_train)
predictions = classifier.predict(x_test_5)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.015509103169251517
Hamming loss  0.04723248459061856
Micro-average quality numbers
Precision: 0.2283, Recall: 0.5062, F1-measure: 0.3147
Macro-average quality numbers
Precision: 0.2706, Recall: 0.5091, F1-measure: 0.2911
              precision    recall  f1-score   support

           0       0.08      0.06      0.07        35
           1       0.38      0.44      0.41       117
           2       0.57      1.00      0.72        26
           3       0.01      0.18      0.02        11
           4       0.15      0.69      0.24        13
           5       0.35      0.92      0.51        26
           6       0.09      0.60      0.15        15
           7       0.30      0.31      0.30        75
           8       0.04      0.25      0.06         4
           9       0.43      0.90      0.58        31
          10       0.10      0.88      0.18         8
          11       0.06      0.05      0.06        20
          12       0.38      0.25      0.30        79
         

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_5, y_train)
predictions = classifier.predict(x_test_5)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.07720836142953473
Hamming loss  0.04128004710664527
Micro-average quality numbers
Precision: 0.2440, Recall: 0.4419, F1-measure: 0.3144
Macro-average quality numbers
Precision: 0.2542, Recall: 0.5118, F1-measure: 0.2801
              precision    recall  f1-score   support

           0       0.10      0.26      0.14        35
           1       0.49      0.25      0.33       117
           2       0.72      1.00      0.84        26
           3       0.02      0.27      0.03        11
           4       0.19      0.77      0.30        13
           5       0.25      1.00      0.40        26
           6       0.15      0.93      0.25        15
           7       0.28      0.32      0.30        75
           8       0.05      0.50      0.08         4
           9       0.51      0.97      0.67        31
          10       0.54      0.88      0.67         8
          11       0.02      0.70      0.04        20
          12       0.26      0.29      0.27        79
          

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_5, y_train)
predictions = classifier.predict(x_test_5)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.07879441178425917
Micro-average quality numbers
Precision: 0.1578, Recall: 0.6173, F1-measure: 0.2513
Macro-average quality numbers
Precision: 0.1867, Recall: 0.5947, F1-measure: 0.2395
              precision    recall  f1-score   support

           0       0.13      0.20      0.16        35
           1       0.25      0.54      0.34       117
           2       0.67      1.00      0.80        26
           3       0.01      0.18      0.01        11
           4       0.15      0.69      0.25        13
           5       0.23      0.96      0.38        26
           6       0.06      0.67      0.11        15
           7       0.16      0.51      0.24        75
           8       0.00      0.50      0.01         4
           9       0.27      1.00      0.42        31
          10       0.70      0.88      0.78         8
          11       0.04      0.65      0.07        20
          12       0.16      0.35      0.22        79
          13       0.02   

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.0001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_5, y_train)
predictions = classifier.predict(x_test_5)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.009777478084962913
Hamming loss  0.05204287084611513
Micro-average quality numbers
Precision: 0.2053, Recall: 0.4980, F1-measure: 0.2908
Macro-average quality numbers
Precision: 0.2024, Recall: 0.5379, F1-measure: 0.2478
              precision    recall  f1-score   support

           0       0.11      0.17      0.13        35
           1       0.20      0.56      0.30       117
           2       0.67      1.00      0.80        26
           3       0.02      0.27      0.04        11
           4       0.12      0.92      0.21        13
           5       0.31      1.00      0.47        26
           6       0.11      0.93      0.19        15
           7       0.19      0.48      0.28        75
           8       0.00      0.25      0.01         4
           9       0.29      1.00      0.45        31
          10       0.07      0.88      0.13         8
          11       0.02      0.10      0.04        20
          12       0.26      0.20      0.23        79
         

In [47]:
vectorizer_char = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(1, 1),  max_features=20000)
x_train_char = vectorizer_char.fit_transform(train['cleaned'])
x_test_char = vectorizer_char.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:35.064447


In [48]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_char, y_train)
predictions = classifier.predict(x_test_char)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.38525827927782474
Micro-average quality numbers
Precision: 0.0282, Recall: 0.5076, F1-measure: 0.0534
Macro-average quality numbers
Precision: 0.0259, Recall: 0.4466, F1-measure: 0.0402
              precision    recall  f1-score   support

           0       0.02      0.43      0.03        35
           1       0.07      0.73      0.13       117
           2       0.01      0.92      0.02        26
           3       0.00      0.64      0.01        11
           4       0.01      0.38      0.03        13
           5       0.01      0.65      0.03        26
           6       0.01      0.33      0.02        15
           7       0.03      0.72      0.06        75
           8       0.00      0.25      0.00         4
           9       0.00      0.00      0.00        31
          10       0.00      0.00      0.00         8
          11       0.01      0.55      0.02        20
          12       0.03      0.68      0.07        79
          13       0.00   

In [49]:
vectorizer_char_2 = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(2, 2),  max_features=20000)
x_train_char_2 = vectorizer_char_2.fit_transform(train['cleaned'])
x_test_char_2 = vectorizer_char_2.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:16.949097


In [50]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_char_2, y_train)
predictions = classifier.predict(x_test_char_2)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.16552144966901883
Micro-average quality numbers
Precision: 0.0699, Recall: 0.5466, F1-measure: 0.1239
Macro-average quality numbers
Precision: 0.0435, Recall: 0.3455, F1-measure: 0.0714
              precision    recall  f1-score   support

           0       0.01      0.34      0.03        35
           1       0.12      0.67      0.21       117
           2       0.03      0.38      0.06        26
           3       0.00      0.00      0.00        11
           4       0.01      0.77      0.02        13
           5       0.02      0.15      0.04        26
           6       0.02      0.20      0.03        15
           7       0.05      0.63      0.09        75
           8       0.00      0.00      0.00         4
           9       0.02      0.45      0.03        31
          10       0.03      0.25      0.05         8
          11       0.01      0.10      0.02        20
          12       0.07      0.47      0.12        79
          13       0.01   

In [51]:
vectorizer_char_3 = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(3, 3),  max_features=20000)
x_train_char_3 = vectorizer_char_3.fit_transform(train['cleaned'])
x_test_char_3 = vectorizer_char_3.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:25.395349


In [52]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"),n_jobs = -1)
classifier.fit(x_train_char_3, y_train)
predictions = classifier.predict(x_test_char_3)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0013486176668914363
Hamming loss  0.08431947043013306
Micro-average quality numbers
Precision: 0.1323, Recall: 0.5283, F1-measure: 0.2116
Macro-average quality numbers
Precision: 0.0749, Recall: 0.2914, F1-measure: 0.1091
              precision    recall  f1-score   support

           0       0.02      0.11      0.03        35
           1       0.13      0.78      0.23       117
           2       0.10      0.38      0.16        26
           3       0.05      0.09      0.07        11
           4       0.07      0.31      0.11        13
           5       0.04      0.46      0.07        26
           6       0.03      0.40      0.06        15
           7       0.06      0.52      0.11        75
           8       0.00      0.00      0.00         4
           9       0.05      0.13      0.07        31
          10       0.11      0.50      0.18         8
          11       0.01      0.05      0.01        20
          12       0.06      0.51      0.11        79
        

In [53]:
vectorizer_char_4 = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(4, 4),  max_features=20000)
x_train_char_4 = vectorizer_char_4.fit_transform(train['cleaned'])
x_test_char_4 = vectorizer_char_4.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:39.478225


In [54]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_char_4, y_train)
predictions = classifier.predict(x_test_char_4)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0023600809170600135
Hamming loss  0.06618911038720522
Micro-average quality numbers
Precision: 0.1608, Recall: 0.4956, F1-measure: 0.2429
Macro-average quality numbers
Precision: 0.0868, Recall: 0.2415, F1-measure: 0.1191
              precision    recall  f1-score   support

           0       0.02      0.06      0.03        35
           1       0.16      0.68      0.26       117
           2       0.10      0.27      0.15        26
           3       0.00      0.00      0.00        11
           4       0.19      0.23      0.21        13
           5       0.09      0.23      0.13        26
           6       0.07      0.20      0.11        15
           7       0.07      0.43      0.12        75
           8       0.00      0.00      0.00         4
           9       0.11      0.13      0.12        31
          10       0.25      0.50      0.33         8
          11       0.00      0.00      0.00        20
          12       0.06      0.41      0.11        79
        

In [55]:
vectorizer_char_5 = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(5, 5),  max_features=20000)
x_train_char_5 = vectorizer_char_5.fit_transform(train['cleaned'])
x_test_char_5 = vectorizer_char_5.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:54.500725


In [56]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_char_5, y_train)
predictions = classifier.predict(x_test_char_5)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.004045853000674309
Hamming loss  0.0652536255971432
Micro-average quality numbers
Precision: 0.1637, Recall: 0.4980, F1-measure: 0.2464
Macro-average quality numbers
Precision: 0.0878, Recall: 0.2492, F1-measure: 0.1177
              precision    recall  f1-score   support

           0       0.01      0.03      0.02        35
           1       0.15      0.70      0.25       117
           2       0.12      0.23      0.16        26
           3       0.04      0.09      0.06        11
           4       0.12      0.46      0.18        13
           5       0.08      0.31      0.13        26
           6       0.07      0.20      0.10        15
           7       0.08      0.47      0.13        75
           8       0.00      0.00      0.00         4
           9       0.11      0.16      0.13        31
          10       0.15      0.50      0.24         8
          11       0.03      0.10      0.04        20
          12       0.07      0.37      0.11        79
          

In [57]:
vectorizer_char_6 = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(6, 6),  max_features=20000)
x_train_char_6 = vectorizer_char_6.fit_transform(train['cleaned'])
x_test_char_6 = vectorizer_char_6.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:01:03.154397


In [58]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_char_6, y_train)
predictions = classifier.predict(x_test_char_6)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0016857720836142953
Hamming loss  0.06522275934772492
Micro-average quality numbers
Precision: 0.1621, Recall: 0.4905, F1-measure: 0.2437
Macro-average quality numbers
Precision: 0.0893, Recall: 0.2404, F1-measure: 0.1189
              precision    recall  f1-score   support

           0       0.03      0.09      0.04        35
           1       0.16      0.70      0.26       117
           2       0.11      0.23      0.15        26
           3       0.03      0.09      0.05        11
           4       0.05      0.15      0.08        13
           5       0.08      0.35      0.12        26
           6       0.09      0.27      0.13        15
           7       0.08      0.43      0.13        75
           8       0.00      0.00      0.00         4
           9       0.12      0.13      0.12        31
          10       0.17      0.50      0.26         8
          11       0.03      0.05      0.03        20
          12       0.07      0.33      0.11        79
        

In [59]:
vectorizer_char_8 = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(7, 7),  max_features=20000)
x_train_char_8 = vectorizer_char_8.fit_transform(train['cleaned'])
x_test_char_8 = vectorizer_char_8.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:01:26.100191


In [60]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_char_8, y_train)
predictions = classifier.predict(x_test_char_8)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0020229265003371545
Hamming loss  0.0618963273911846
Micro-average quality numbers
Precision: 0.1687, Recall: 0.4809, F1-measure: 0.2497
Macro-average quality numbers
Precision: 0.0973, Recall: 0.2290, F1-measure: 0.1194
              precision    recall  f1-score   support

           0       0.01      0.03      0.02        35
           1       0.16      0.69      0.26       117
           2       0.16      0.23      0.19        26
           3       0.00      0.00      0.00        11
           4       0.11      0.31      0.16        13
           5       0.09      0.38      0.15        26
           6       0.06      0.20      0.09        15
           7       0.08      0.41      0.14        75
           8       0.00      0.00      0.00         4
           9       0.12      0.16      0.14        31
          10       0.17      0.50      0.25         8
          11       0.00      0.00      0.00        20
          12       0.07      0.32      0.12        79
         

In [61]:
vectorizer_charngram = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(1, 6),  max_features=20000)
x_train_charngram = vectorizer_charngram.fit_transform(train['cleaned'])
x_test_charngram = vectorizer_charngram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:03:24.349316


In [62]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_charngram, y_train)
predictions = classifier.predict(x_test_charngram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.004045853000674309
Hamming loss  0.08361666967414738
Micro-average quality numbers
Precision: 0.1341, Recall: 0.5318, F1-measure: 0.2141
Macro-average quality numbers
Precision: 0.0731, Recall: 0.3132, F1-measure: 0.1114
              precision    recall  f1-score   support

           0       0.02      0.11      0.04        35
           1       0.14      0.74      0.23       117
           2       0.05      0.42      0.08        26
           3       0.02      0.27      0.03        11
           4       0.06      0.31      0.11        13
           5       0.04      0.42      0.07        26
           6       0.03      0.33      0.05        15
           7       0.08      0.55      0.14        75
           8       0.00      0.00      0.00         4
           9       0.04      0.19      0.07        31
          10       0.08      0.50      0.14         8
          11       0.02      0.15      0.04        20
          12       0.06      0.46      0.11        79
         

In [63]:
vectorizer_charngram = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char', ngram_range=(2, 4),  max_features=20000)
x_train_charngram = vectorizer_charngram.fit_transform(train['cleaned'])
x_test_charngram = vectorizer_charngram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:01:45.282984


In [64]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_charngram, y_train)
predictions = classifier.predict(x_test_charngram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0006743088334457181
Hamming loss  0.07980824936130607
Micro-average quality numbers
Precision: 0.1383, Recall: 0.5211, F1-measure: 0.2186
Macro-average quality numbers
Precision: 0.0789, Recall: 0.2743, F1-measure: 0.1131
              precision    recall  f1-score   support

           0       0.02      0.09      0.03        35
           1       0.16      0.71      0.27       117
           2       0.08      0.38      0.14        26
           3       0.05      0.27      0.09        11
           4       0.04      0.38      0.08        13
           5       0.03      0.38      0.06        26
           6       0.04      0.20      0.06        15
           7       0.07      0.47      0.12        75
           8       0.00      0.00      0.00         4
           9       0.05      0.13      0.08        31
          10       0.18      0.50      0.27         8
          11       0.02      0.15      0.04        20
          12       0.07      0.48      0.12        79
        

In [65]:
x_train_1 = hstack((x_train_multilabel_bigram, x_train_char_5),format="csr",dtype='float64')

In [66]:
x_test_1 = hstack((x_test_multilabel_bigram, x_test_char_5),format="csr",dtype='float64')

In [67]:
train = x_train_1.toarray()

In [68]:
test = x_test_1.toarray()

In [69]:
x_train_6 = np.hstack((train, train_glove))

In [70]:
x_test_6 = np.hstack((test, test_glove))

In [ ]:
np.save('final_train_for_every_important_feature.npy', x_train_6)
np.save('final_test_for_every_important_feature.npy', x_test_6)

In [71]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs = -1)
classifier.fit(x_train_6, y_train)
predictions = classifier.predict(x_test_6)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0050573162508428865
Hamming loss  0.08043744598406352
Micro-average quality numbers
Precision: 0.1693, Recall: 0.7054, F1-measure: 0.2731
Macro-average quality numbers
Precision: 0.1557, Recall: 0.6540, F1-measure: 0.2236
              precision    recall  f1-score   support

           0       0.05      0.69      0.10        35
           1       0.18      0.76      0.30       117
           2       0.76      1.00      0.87        26
           3       0.01      0.27      0.02        11
           4       0.11      0.85      0.20        13
           5       0.19      1.00      0.31        26
           6       0.08      0.80      0.15        15
           7       0.12      0.72      0.21        75
           8       0.01      0.25      0.01         4
           9       0.24      1.00      0.38        31
          10       0.17      0.88      0.29         8
          11       0.03      0.45      0.05        20
          12       0.07      0.76      0.13        79
        

_Let us try topic modelling_